# Elem Example Sentence Prompt

In [1]:
import pandas as pd
from openai import OpenAI
from jinja2 import Template
from pydantic import BaseModel
from openai.lib._parsing import type_to_response_format_param
import json
from getpass import getpass

In [2]:
openai_api_key = getpass("OPENAI_API_KEY")
client = OpenAI(api_key=openai_api_key)

# 초등 prompt 설정

In [6]:
prompt_voca_sentence_elem_template = Template("""
[PERSONA]
You are going to be given a set of English words and some of their Korean meaning. As an elementary school English teacher, generate English sentences using each of those words, and provide Korean translation, too. You must refer to the given Korean meaning to generate the examples. Overall sentences must be very easy and kids-friendly. You must refer to the given Korean meaning to generate the examples.


[POLICY]
1. A sentence must not be over 6 words.
2. Separate the sentence with a tab in between.
3. For Korean translation, use '~요' at the end of each translated sentence to soften the tone. Make it natural.
4. The whole data must strictly follow CEFR A1 level.

[EXAMPLE]
```
Input:
    {
    "voca_NO" : 123
    "WordText" : "cat",
    "WordMeaning" : "고양이",
    "PoSText" : "명사"
    }

Output:
    {
     "example_sentence" : "The cat sleeps on the bed.",
     "example_sentence_kor" : "고양이가 침대에서 자요."
    }
```                           
                           
[Input]
```
voca_NO : {{voca_NO}}
WordText : {{WordText}}
WordMeaning : {{WordMeaning}}
PoSText : {{PoSText}}
```                   
"""                                
)


- 예시 출력

In [18]:
prompt_voca_sentence_elem = prompt_voca_sentence_elem_template.render(
    voca_NO = "123",
    WordText = "everyone",
    WordMeaning = "모든 사람",
    PoSText = "대명사"
)

In [27]:
print(prompt_voca_sentence_elem)


[PERSONA]
You are going to be given a set of English words and some of their Korean meaning. As an elementary school English teacher, generate English sentences using each of those words, and provide Korean translation, too. You must refer to the given Korean meaning to generate the examples. Overall sentences must be very easy and kids-friendly. You must refer to the given Korean meaning to generate the examples.


[POLICY]
1. A sentence must not be over 6 words.
2. Separate the sentence with a tab in between.
3. For Korean translation, use '~요' at the end of each translated sentence to soften the tone. Make it natural.
4. The whole data must strictly follow CEFR A1 level.

[EXAMPLE]
```
Input:
    {
    "voca_NO" : 123
    "WordText" : "cat",
    "WordMeaning" : "고양이",
    "PoSText" : "명사"
    }

Output:
    {
     "example_sentence" : "The cat sleeps on the bed.",
     "example_sentence_kor" : "고양이가 침대에서 자요."
    }
```                           
                           
[Input]


In [28]:
class ExampleSentence(BaseModel):
    example_sentence : str
    example_sentence_kor : str

In [10]:
response_format = type_to_response_format_param(ExampleSentence)

In [11]:
response_format

{'type': 'json_schema',
 'json_schema': {'schema': {'properties': {'example_sentence': {'title': 'Example Sentence',
     'type': 'string'},
    'example_sentence_kor': {'title': 'Example Sentence Kor',
     'type': 'string'}},
   'required': ['example_sentence', 'example_sentence_kor'],
   'title': 'ExampleSentence',
   'type': 'object',
   'additionalProperties': False},
  'name': 'ExampleSentence',
  'strict': True}}

In [29]:
def completion(prompt : str, response_format : str) -> str:
    response = client.beta.chat.completions.parse(
        model = 'o3-mini',
        reasoning_effort = 'low',
        messages = [
            {"role" : "system", "content" : "Generate easy example sentences."},
            {"role" : "user", "content" : prompt}
        ],
        response_format = response_format
    )
    return response.choices[0].message.parsed

In [30]:
response = completion(prompt_voca_sentence_elem, response_format = ExampleSentence)

In [31]:
print(response)

example_sentence='Everyone plays in the park.' example_sentence_kor='모든 사람이 공원에서 놀아요.'


In [32]:
response_output = json.dumps(response.dict(), ensure_ascii=False, indent=4)
print(response_output)

{
    "example_sentence": "Everyone plays in the park.",
    "example_sentence_kor": "모든 사람이 공원에서 놀아요."
}


# data 추가

In [37]:
file_name = "Final_Voca_Elem.csv"
df = pd.read_csv(file_name)
df.head(5)

,textbook ID,학년,출판사(저자),교육과정,레슨,단어,뜻,예문,예문 해석,영영풀이,초등800 포함여부
0,301,초3,천재(함),2015,1,bye,int. 안녕\nn.부전승,NaN,NaN,NaN,True
1,301,초3,천재(함),2015,1,hi,int.안녕,NaN,NaN,NaN,True
2,301,초3,천재(함),2015,1,hello,"pl. 만났을때 인사, 안녕하세요",NaN,NaN,NaN,True
3,301,초3,천재(함),2015,1,I,"n.(알파벳)아이, (로마숫자)1",NaN,NaN,NaN,True
4,301,초3,천재(함),2015,1,sorry,"a.안된, 미안한",NaN,NaN,NaN,True


# jsonl 파일 변환

In [26]:
def VocaQuizMake(data, output_filename):
    jsonl_data = []

    for i in range(len(data)):
        prompt = prompt_voca_sentence_elem_template.render(
            id = data.loc['voca NO'],
            word = data.loc[i,"WordText"],
            meaning = data.loc[i,"WordMeaning"],
            part_of_speech = data.loc[i,"품사"]
        )

        quiz_request = {
            "custom_id" : f"request-{i+1}",
            "method" : "POST",
            "url" : "/v1/chat/completions",
            "body" : {
                "model" : "o3-mini-2025-01-31",
                "messages" : [
                    {"role": "system", "content": "Generate easy example sentences."},
                    {"role": "user", "content": prompt}
                ],
                "response_format" : response_format
            }
        }

        jsonl_data.append(quiz_request)

        with open(output_filename, 'w', encoding='utf-8') as jsonl_file:
            for item in jsonl_data:
                jsonl_file.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f'JSONL 파일 생성 완료 : {output_filename}-{i+1}')